In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import pymongo

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw
from climatools.cliradlw.utils import mapband_new2old

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *

import IPython.display as display

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
def make_query(param=None):
    return {'param.' + name: value for name, value in param.items()}   

In [4]:
def show_html(s): display.display(display.HTML(s))
    
def show_markdown(s): display.display(display.Markdown(s))

In [5]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['i', 'band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [6]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    dims = ['igg', 'g', 'i']
    for dim in dims:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')     
    return ds['coolrg']

def pltdata_cooling_1():
    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(DS_COOL.sel(i=1)),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(DS_COOL_CRD.sum('g')),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(DS_COOL_WGT.sel(igg=10).sum('g')),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data

def nice_xlims(pltdata=None, prange=None):
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)

def plt_cool(pltdata=None):
    plin = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='linear',
                                  prange=(50, 1050))
    plin.plot_width, plin.plot_height = 400, 500
    plog = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='log',
                                  prange=(.01, 200))
    plog.plot_width, plog.plot_height = 400, 500
    everything = gridplot(plin, plog, ncols=2)
    return everything
    
def script_plt_cooling():
    pltdata = pltdata_cooling_1()
    p = plt_cool(pltdata=pltdata)
    show(p)
    show_markdown('''*FIGURE.* Cooling rate profiles.''')

In [7]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels: ds = ds.isel(pressure=ilevels)

    dims = ['i', 'igg', 'g']
    for dim in dims:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df

def tbdata_flux():
    tropopauses = {'mls': 40, 'saw': 45, 'trp': 37}
    atmpro = PARAM['atmpro']
    ilevels = [0, tropopauses[atmpro], -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(DS_FLUX_CRD.sum('g'), ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10',
            'df': fmt_flux(DS_FLUX_WGT.sum('g').sel(igg=10), 
                           ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(DS_FLUX.sel(i=1), ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data

def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        show_markdown(f'''*TABLE.*  Fluxes. {d['label']}''')
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        show_markdown(
            f'''*TABLE.*  Fluxes.  ({d['label']}) - ({d0['label']})''')
        
    show_tb(benchmark)
    for d in others: show_tb(d)
    for d in others: show_tbdiff(d=d, d0=benchmark)
        
def script_tb_flux():
    data = tbdata_flux()
    tb_flux(tbdata=data)

In [8]:
def gasbands():
    bmap = mapband_new2old()
    gases = [gas for gas, _ in PARAM['molecule'].items()] 
    bands = [bmap[b] for b in PARAM['band']]
    return list(itertools.product(gases, bands))

def print_bestfit_params():
    df = pd.DataFrame()
    for gas, band in gasbands():
        param = bestfits.kdist_params(molecule=gas, band=band)
        srs = pd.Series(param)
        df[f'({gas}, {band})'] = srs
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df.sort_index())
    pd.reset_option('display.max_colwidth')
        
def print_input_param():
    df_clirad = pd.Series(PARAM, name='clirad run').to_frame()
    df_lblnew = pd.Series(PARAM_LBLNEW, name='lblnew run compared against').to_frame()
    df = pd.merge(df_clirad, df_lblnew, how='outer', 
                  left_index=True, right_index=True)
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df)
    pd.reset_option('display.max_colwidth')
        
def print_lblnew_param():
    pd.set_option('display.max_colwidth', -1)
    display.display(pd.Series(PARAM_LBLNEW).to_frame())
    pd.reset_option('display.max_colwidth')

In [9]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD.sum('g'), DS_COOL.sel(i=1)]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [10]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD.sum('g'), DS_FLUX.sel(i=1)]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [11]:
def script_fluxcoolr_excelfile():
    '''
    Save fluxes and cooling rates by layer by g-interval in excel file
    and make it available for download.
    '''
    fname = 'output_byg.xlsx'
    
    writer = pd.ExcelWriter(fname)
    
    for g in DS_FLUX_WGT.coords['g']:
        df = DS_FLUX_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'flux g={int(g)}')
        
    for g in DS_COOL_WGT.coords['g']:
        df = DS_COOL_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'cool g={int(g)}')    

In [12]:


    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()

In [13]:
%run param.py

client = pymongo.MongoClient('localhost', 27017)

doc = client.lblnew.bestfit_lw.find_one(make_query(param=PARAM_LBLNEW))
DS_FLUX_CRD = load_output_file(io.StringIO(doc['output_fluxg']))
DS_COOL_CRD = load_output_file(io.StringIO(doc['output_coolrg']))
DS_FLUX_WGT = load_output_file(io.StringIO(doc['output_wfluxg']))
DS_COOL_WGT = load_output_file(io.StringIO(doc['output_wcoolrg']))

doc = client.cliradnew.lw.find_one(make_query(param=PARAM))
DS_FLUX = load_output_file(io.StringIO(doc['output_flux']))
DS_COOL = load_output_file(io.StringIO(doc['output_coolr']))

In [14]:
script()
if 'ng_refs' in PARAM_LBLNEW:
    script_fluxcoolr_excelfile()

### CLIRAD-LW

### Table of Contents

# Input Parameters

,clirad run,lblnew run compared against
atmpro,mls,mls
band,[3],3a
commitnumber,1013f91,a22ab94
conc,-,0.0004
dv,-,0.001
klin,-,6.37556e-24
molecule,"{'co2': 0.0004, 'h2o': 'atmpro', 'n2o': 3.2e-07}",co2
ng_adju,-,"[0, 0, 0]"
ng_refs,-,"[3, 2, 4]"
nv,-,1000


# Best-fit Parameters

...

*TABLE.*  Fluxes.  (CLIRAD) - (CRD)

# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312        1  0.389489                    1  0.250868
0.000750        2  0.388176                    2  0.254957
0.001052        3  0.399332                    3  0.265913
0.001476        4  0.409658                    4  0.277447
0.002070        5  0.417947                    5  0.288475
0.002904        6  0.422956                    6  0.298888
0.004074        7  0.424201                    7  0.308031
0.005714        8  0.420278                    8  0.315538
0.008015        9  0.409925                    9  0.320296
0.011243       10  0.391847                   10  0.321429
0.015771       11  0.364466                   11  0.317100
0.022122       12  0.326236                   12  0.304946
0.031031       13  0.275434                   13  0.282233
0.043528       14  0.210205                   14  0.245880
0.061057       15  0.160402                   15  0.220136
0.085645       16  0.178620                   16  0.249182
0.120136       17  0.241901                   17  0.311268
0.168516       18  0.291570                   18  0.347836
0.236378       19  0.331875                   19  0.357865
0.331549       20  0.378121                   20  0.371456
0.465100       21  0.448609                   21  0.431794
0.652400       22  0.574815                   22  0.576037
0.915100       23  0.721690                   23  0.746603
1.283650       24  0.691988                   24  0.741398
1.800600       25  0.535288                   25  0.597912
2.525700       26  0.414367                   26  0.474079
3.542800       27  0.325918                   27  0.389355
4.969550       28  0.257415                   28  0.325800
6.970850       29  0.205556                   29  0.273654
9.778100       30  0.164672                   30  0.220863
13.715850      31  0.127783                   31  0.166986
19.239350      32  0.094104                   32  0.116454
26.987250      33  0.071226                   33  0.081808
37.855300      34  0.057832                   34  0.064855
53.100050      35  0.044322                   35  0.054382
73.887500      36  0.029910                   36  0.042427
97.662500      37  0.021253                   37  0.032940
121.437500     38  0.016832                   38  0.026023
145.212500     39  0.008260                   39  0.014095
168.987500     40 -0.001416                   40  0.001963
192.762500     41  0.000408                   41  0.004677
216.537500     42  0.013739                   42  0.024842
240.312500     43  0.025656                   43  0.049383
264.087500     44  0.035534                   44  0.075665
287.862500     45  0.044365                   45  0.101304
311.637500     46  0.052432                   46  0.118706
335.412500     47  0.059858                   47  0.134376
359.187500     48  0.066717                   48  0.148939
382.962500     49  0.073051                   49  0.162665
406.737500     50  0.078825                   50  0.177827
430.512500     51  0.083967                   51  0.195732
454.287500     52  0.088361                   52  0.212506
478.062500     53  0.091971                   53  0.227753
501.837500     54  0.094914                   54  0.241318
525.612500     55  0.097388                   55  0.254054
549.387500     56  0.099557                   56  0.266276
573.162500     57  0.101520                   57  0.278849
596.937500     58  0.103342                   58  0.300264
620.712500     59  0.105074                   59  0.319973
644.487500     60  0.106757                   60  0.335295
668.262500     61  0.108432                   61  0.345325
692.037500     62  0.110170                   62  0.349313
715.812500     63  0.112058                   63  0.348991
739.587500     64  0.114182                   64  0.345562
763.362500     65  0.116627                   65  0.339896
78

# Fluxes by Layer

CRD                           CLIRAD                \
                        flug       fldg      fnetg       flug          fldg   
pressure    level                                                             
0.000000    1     -29.581934   0.000000 -29.581934 -25.175149  4.415289e-08   
0.000624    2     -29.581952   0.000046 -29.581905 -25.175166  3.585924e-05   
0.000876    3     -29.581958   0.000064 -29.581894 -25.175173  5.038747e-05   
0.001229    4     -29.581968   0.000091 -29.581877 -25.175183  7.113339e-05   
0.001723    5     -29.581981   0.000128 -29.581853 -25.175196  1.007820e-04   
0.002417    6     -29.581999   0.000180 -29.581819 -25.175215  1.430959e-04   
0.003391    7     -29.582025   0.000254 -29.581770 -25.175241  2.034099e-04   
0.004757    8     -29.582059   0.000358 -29.581701 -25.175277  2.892007e-04   
0.006672    9     -29.582108   0.000502 -29.581606 -25.175327  4.108943e-04   
0.009359    10    -29.582176   0.000700 -29.581476 -25.175397  5.828260e-04   
0.013128    11    -29.582270   0.000970 -29.581301 -25.175495  8.245088e-04   
0.018415    12    -29.582404   0.001332 -29.581072 -25.175634  1.161797e-03   
0.025830    13    -29.582595   0.001810 -29.580786 -25.175831  1.627423e-03   
0.036232    14    -29.582874   0.002428 -29.580446 -25.176117  2.260896e-03   
0.050823    15    -29.583295   0.003211 -29.580083 -25.176538  3.106647e-03   
0.071291    16    -29.583932   0.004238 -29.579694 -25.177156  4.258631e-03   
0.100000    17    -29.584849   0.005762 -29.579086 -25.178017  5.967343e-03   
0.140271    18    -29.586078   0.008145 -29.577933 -25.179136  8.571396e-03   
0.196760    19    -29.587672   0.011691 -29.575981 -25.180531  1.229414e-02   
0.275997    20    -29.589660   0.016793 -29.572867 -25.182181  1.730347e-02   
0.387100    21    -29.591969   0.024080 -29.567890 -25.183957  2.396912e-02   
0.543100    22    -29.594254   0.034654 -29.559600 -25.185461  3.345237e-02   
0.761700    23    -29.595106   0.050390 -29.544716 -25.185294  4.820348e-02   
1.068500    24    -29.591321   0.072834 -29.518487 -25.180525  7.057050e-02   
1.498800    25    -29.582270   0.099055 -29.483215 -25.170651  9.849140e-02   
2.102400    26    -29.570761   0.125820 -29.444942 -25.158041  1.286369e-01   
2.949000    27    -29.557826   0.154439 -29.403386 -25.143546  1.616900e-01   
4.136600    28    -29.543780   0.186244 -29.357537 -25.127287  2.002110e-01   
5.802500    29    -29.528986   0.222247 -29.306739 -25.109481  2.467046e-01   
8.139200    30    -29.513547   0.263706 -29.249842 -25.090007  3.029854e-01   
11.417000   31    -29.497520   0.311618 -29.185903 -25.068991  3.677349e-01   
16.014700   32    -29.482221   0.365913 -29.116308 -25.048144  4.378431e-01   
22.464000   33    -29.471270   0.426855 -29.044416 -25.032217  5.108926e-01   
31.510500   34    -29.467531   0.499443 -28.968088 -25.025348  5.916996e-01   
44.200100   35    -29.472291   0.591134 -28.881157 -25.029822  6.936729e-01   
62.000000   36    -29.493049   0.705345 -28.787704 -25.055680  8.342094e-01   
85.775000   37    -29.545127   0.841658 -28.703468 -25.122427  1.020456e+00   
109.550000  38    -29.617933   0.974322 -28.643612 -25.215567  1.206376e+00   
133.325000  39    -29.706814   1.110606 -28.596207 -25.328634  1.392741e+00   
157.100000  40    -29.818628   1.245682 -28.572945 -25.468185  1.571992e+00   
180.875000  41    -29.964343   1.387409 -28.576934 -25.644558  1.753894e+00   
204.650000  42    -30.139695   1.563911 -28.575785 -25.849752  1.972260e+00   
228.425000  43    -30.329752   1.792660 -28.537091 -26.070615  2.263092e+00   
252.200000  44    -30.529194   2.064357 -28.464836 -26.304179  2.635749e+00   
275.975000  45    -30.736253   2.371491 -28.364761 -26.552877  3.097566e+00   
299.750000  46    -30.949555   2.709740 -28.239815 -26.820585  3.650608e+00   
323.525000  47    -31.167843   3.075694 -28.092149 -27.101239  4.265612e+00   
347.300000  48    -31.389935   3.466366 -27.923569 -27.391160  4.934017e+00

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')